In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("/Users/sarah/ML/master_thesis/Datasets/shuffled_data.csv")# Load mapping and clinical data
id_map = pd.read_csv("/Users/sarah/ML/master_thesis/Datasets/patient_id_mapping.csv")
id_map.columns = ['Anon.Patient_ID', 'study_id']
clinical = pd.read_excel("/Users/sarah/Library/Mobile Documents/iCloud~md~obsidian/Documents/MSc Medizininformatik/Master Thesis/Data/219 patients.xlsx")

# Merge measurement data with mapping
df_with_id = df.merge(id_map, on="Anon.Patient_ID", how="left")

# Merge with clinical data
df_full = df_with_id.merge(clinical, left_on="study_id", right_on="study_id", how="left")

df= df_full.drop(columns=["study_id", "Anon.Patient_ID"])

In [20]:
clinical = pd.read_excel("/Users/sarah/Library/Mobile Documents/iCloud~md~obsidian/Documents/MSc Medizininformatik/Master Thesis/Data/219 patients.xlsx")

# Load the z-score dataset (adjust path and column name as needed)
bwz_df = pd.read_excel("/Users/sarah/ML/master_thesis/Clinical Data Model/Thesis Writing/219 patients_bw.xlsx")

# Make sure the column names align
# Example: bwz_df.columns = ['Anon.Patient_ID', 'study_id', 'Birthweight_Z']

# Merge with patient_id_mapping (if needed)
# if 'study_id' not in bwz_df.columns:
#     bwz_df = bwz_df.merge(id_map, on="Anon.Patient_ID", how="left")

# Merge with main df_full
df_full = clinical.merge(bwz_df[['study_id', 'Birth_wt_z']], 
                        on=["study_id"], how="left")

# Done! You now have 'Birthweight_Z' added to df_full


In [21]:
import pandas as pd

df = df_full

# Updated numerical feature names from your dataset
numerical_features = [
    "Birth_wt_z",        # Birthweight z-score
    "GA_wd",             # Gestational age
    "PMA_test_w",        # Postmenstrual age at test
    "Hb_at_test",        # Hemoglobin at time of test
    "shift_2C",          # Shift (2-compartment)
    "shunt_2C"           # Shunt (2-compartment)
]

# Display-friendly names for the table
display_names = {
    "Birth_wt_z": "Birthweight z-score",
    "GA_wd": "Gestational Age (weeks)",
    "PMA_test_w": "Postmenstrual Age at Test (weeks)",
    "Hb_at_test": "Hemoglobin at Test (g/dL)",
    "shift_2C": "Shift (2C model)",
    "shunt_2C": "Shunt (2C model)"
}

# Create summary
num_summary = df[numerical_features].describe().T
num_summary["missing"] = df[numerical_features].isnull().sum()
num_summary["IQR"] = df[numerical_features].quantile(0.75) - df[numerical_features].quantile(0.25)
num_summary = num_summary[["count", "mean", "std", "min", "25%", "50%", "75%", "max", "IQR", "missing"]].round(2)

# Rename for presentation
num_summary.rename(index=display_names, inplace=True)

# Export and print
num_summary.to_csv("renamed_numerical_summary.csv")
print(num_summary.to_latex())


\begin{tabular}{lrrrrrrrrrr}
\toprule
 & count & mean & std & min & 25% & 50% & 75% & max & IQR & missing \\
\midrule
Birthweight z-score & 219.000000 & 0.070000 & 0.850000 & -2.370000 & -0.440000 & 0.030000 & 0.640000 & 2.140000 & 1.080000 & 0 \\
Gestational Age (weeks) & 219.000000 & 27.830000 & 2.340000 & 23.000000 & 26.000000 & 27.860000 & 29.570000 & 31.860000 & 3.570000 & 0 \\
Postmenstrual Age at Test (weeks) & 219.000000 & 35.410000 & 0.890000 & 33.710000 & 34.710000 & 35.430000 & 36.000000 & 39.290000 & 1.290000 & 0 \\
Hemoglobin at Test (g/dL) & 219.000000 & 104.180000 & 22.470000 & 9.000000 & 92.000000 & 103.000000 & 115.000000 & 199.000000 & 23.000000 & 0 \\
Shift (2C model) & 219.000000 & 11.940000 & 3.870000 & 5.250000 & 9.740000 & 11.050000 & 12.540000 & 39.450000 & 2.800000 & 0 \\
Shunt (2C model) & 219.000000 & 5.080000 & 6.540000 & 0.000000 & 0.000000 & 3.280000 & 7.740000 & 52.450000 & 7.740000 & 0 \\
\bottomrule
\end{tabular}



In [22]:
import pandas as pd
from scipy.stats import mannwhitneyu, kruskal

# Feature groups
numerical_features = [
    "Birth_wt_z", "GA_wd", "PMA_test_w", "Hb_at_test", "shift_2C", "shunt_2C"
]
categorical_features = ["sex", "BPD_yn", "BPD_severity"]

display_names = {
    "Birth_wt_z": "Birthweight z-score",
    "GA_wd": "Gestational Age (weeks)",
    "PMA_test_w": "Postmenstrual Age at Test (weeks)",
    "Hb_at_test": "Hemoglobin at Test (g/dL)",
    "shift_2C": "Shift (2C model)",
    "shunt_2C": "Shunt (2C model)"
}

# Numerical summary
num_summary = df[numerical_features].describe().T
num_summary["missing"] = df[numerical_features].isnull().sum()
num_summary["IQR"] = df[numerical_features].quantile(0.75) - df[numerical_features].quantile(0.25)

# Stats by BPD_yn (binary)
mw_results = []
kw_results = []
for col in numerical_features:
    group1 = df[df["BPD_yn"] == 0][col].dropna()
    group2 = df[df["BPD_yn"] == 1][col].dropna()
    if len(group1) > 0 and len(group2) > 0:
        stat, p = mannwhitneyu(group1, group2, alternative="two-sided")
    else:
        stat, p = (None, None)
    mw_results.append((stat, p))
    
    # Kruskal-Wallis for severity
    groups = [grp[col].dropna() for _, grp in df.groupby("BPD_severity")]
    if all(len(g) > 0 for g in groups):
        k_stat, k_p = kruskal(*groups)
    else:
        k_stat, k_p = (None, None)
    kw_results.append((k_stat, k_p))

# Add test results to table
num_summary["MW_U_BPD"] = [f"{s:.2f}" if s is not None else "-" for s, _ in mw_results]
num_summary["p_BPD"] = [f"{p:.4f}" if p is not None else "-" for _, p in mw_results]
num_summary["KW_H_Severity"] = [f"{s:.2f}" if s is not None else "-" for s, _ in kw_results]
num_summary["p_Severity"] = [f"{p:.4f}" if p is not None else "-" for _, p in kw_results]

# Rename for display
num_summary.rename(index=display_names, inplace=True)
num_summary = num_summary.round(2)

# Categorical summary
cat_summary = []
for col in categorical_features:
    counts = df[col].value_counts(dropna=False).sort_index()
    total = counts.sum()
    for cat, count in counts.items():
        percent = (count / total) * 100
        cat_summary.append({
            "feature": col,
            "category": cat,
            "count": count,
            "percent": round(percent, 1)
        })
cat_summary_df = pd.DataFrame(cat_summary)

# Save and export
num_summary.to_csv("numerical_summary_with_tests.csv")
cat_summary_df.to_csv("categorical_summary.csv")

# Print for Overleaf
print("=== Numerical Summary with Statistical Tests ===")
print(num_summary.to_latex())

print("\n=== Categorical Summary ===")
print(cat_summary_df.to_latex(index=False))


=== Numerical Summary with Statistical Tests ===
\begin{tabular}{lrrrrrrrrrrllll}
\toprule
 & count & mean & std & min & 25% & 50% & 75% & max & missing & IQR & MW_U_BPD & p_BPD & KW_H_Severity & p_Severity \\
\midrule
Birthweight z-score & 219.000000 & 0.070000 & 0.850000 & -2.370000 & -0.440000 & 0.030000 & 0.640000 & 2.140000 & 0 & 1.080000 & 4690.50 & 0.0248 & 6.89 & 0.0753 \\
Gestational Age (weeks) & 219.000000 & 27.830000 & 2.340000 & 23.000000 & 26.000000 & 27.860000 & 29.570000 & 31.860000 & 0 & 3.570000 & 10533.00 & 0.0000 & 111.02 & 0.0000 \\
Postmenstrual Age at Test (weeks) & 219.000000 & 35.410000 & 0.890000 & 33.710000 & 34.710000 & 35.430000 & 36.000000 & 39.290000 & 0 & 1.290000 & 2855.00 & 0.0000 & 43.97 & 0.0000 \\
Hemoglobin at Test (g/dL) & 219.000000 & 104.180000 & 22.470000 & 9.000000 & 92.000000 & 103.000000 & 115.000000 & 199.000000 & 0 & 23.000000 & 4532.00 & 0.0095 & 9.83 & 0.0201 \\
Shift (2C model) & 219.000000 & 11.940000 & 3.870000 & 5.250000 & 9.740000 &

In [26]:
import pandas as pd
from scipy.stats import mannwhitneyu, kruskal

# --- Load your final merged dataframe ---
df = df_full

# --- Define features ---
numerical_features = {
    "Birth_wt_z": "Birthweight z-score",
    "GA_wd": "Gestational Age (weeks)",
    "PMA_test_w": "Postmenstrual Age at Test (weeks)",
    "Hb_at_test": "Hemoglobin at Test (g/dL)",
    "shift_2C": "Shift (2C model)",
    "shunt_2C": "Shunt (2C model)"
}

bpd_col = "BPD_yn"
severity_col = "BPD_severity"

# --- Build summary table ---
summary = []

for col, label in numerical_features.items():
    data = df[col].dropna()
    desc = data.describe()
    iqr = data.quantile(0.75) - data.quantile(0.25)
    missing = df[col].isnull().sum()

    # Group tests
    try:
        g1 = df[df[bpd_col] == 0][col].dropna()
        g2 = df[df[bpd_col] == 1][col].dropna()
        mw_stat, mw_p = mannwhitneyu(g1, g2)
    except:
        mw_stat, mw_p = (None, None)

    try:
        groups = [grp[col].dropna() for _, grp in df.groupby(severity_col)]
        kw_stat, kw_p = kruskal(*groups)
    except:
        kw_stat, kw_p = (None, None)

  # ... same as before ...

    summary.append({
        "Feature": label,
        "N": int(desc["count"]),
        "Mean": round(desc["mean"], 2),
        "SD": round(desc["std"], 2),
        "Median": round(desc["50%"], 2),
        "IQR": round(iqr, 2),
        "Missing": missing,
        "MW-U (BPD)": f"{mw_stat:.1f}" if mw_stat else "-",
        "p (BPD)": "< 0.001" if mw_p is not None and mw_p < 0.001 else (f"{mw_p:.4f}" if mw_p is not None else "-"),
        "H (Severity)": f"{kw_stat:.1f}" if kw_stat else "-",
        "p (Severity)": "< 0.001" if kw_p is not None and kw_p < 0.001 else (f"{kw_p:.4f}" if kw_p is not None else "-")
    })

summary_df = pd.DataFrame(summary)

# --- Save to Excel for import into Word ---
summary_df.to_excel("ODC_summary_table.xlsx", index=False)


In [25]:

# Define features with label mappings (optional)
features = {
    "sex": {0: "female", 1: "male"},
    "BPD_yn": {0: "no", 1: "yes"},
    "BPD_severity": {
        0: "0",
        1: "1",
        2: "2",
        3: "3"
    }
}

# Build long-format table
rows = []

for feature, mapping in features.items():
    total = df[feature].notna().sum()
    value_counts = df[feature].value_counts(dropna=False).sort_index()

    for code, label in mapping.items():
        count = value_counts.get(code, 0)
        percent = round((count / total) * 100, 2)
        rows.append({
            "Feature": feature.replace("_yn", "").replace("_", " ").title(),
            "Value": label,
            "amount": count,
            "percent": f"{percent} %"
        })

# Create and export
long_cat_summary = pd.DataFrame(rows)
long_cat_summary.to_excel("ODC_categorical_summary_long.xlsx", index=False)

# Preview
print(long_cat_summary)

        Feature   Value  amount  percent
0           Sex  female     138  63.01 %
1           Sex    male      81  36.99 %
2           Bpd      no     133  60.73 %
3           Bpd     yes      86  39.27 %
4  Bpd Severity       0     133  60.73 %
5  Bpd Severity       1      31  14.16 %
6  Bpd Severity       2      22  10.05 %
7  Bpd Severity       3      33  15.07 %
